In [1]:
# !echo $RAYCLUSTER_AUTOSCALER_HEAD_SVC_PORT_10001_TCP_ADDR
# !echo $RAYCLUSTER_AUTOSCALER_HEAD_SVC_PORT_10001_TCP_PORT

In [6]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

In [7]:
import ray
import time

In [8]:
# from utils import logger

In [1]:
%load_ext autoreload
%autoreload 2

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score


from utils import helper, config, rayer, kaggle_dataset_helper
from ml.models.base.v2.brisk_xgboost import BriskXGBoost
from ml.models.base.v2.brisk_bagging import BriskBagging
from ml.models.base.v2.slug_xgboost import SlugXGBoost
from ml.models.base.v2.slug_knn import SlugKNN
from ml.models.base.v2.slug_rf import SlugRF


from ml.models import common


ModuleNotFoundError: No module named 'utils'

In [10]:
rayer.get_global_cluster(num_cpus=4)

In [11]:
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

======== Autoscaler status: 2022-11-22 13:12:06.009202 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 9 small-group
Pending:
 IP not yet assigned: small-group, waiting
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 0.0/19.0 CPU
 0.00/68.918 GiB memory
 0.00/20.368 GiB object_store_memory

Demands:
 {'CPU': 1}: 4+ from request_resources()


In [8]:
ray.autoscaler.sdk.request_resources(num_cpus=0)

In [16]:
ds_train, ds_test = kaggle_dataset_helper.get_transaction_predictions_dataset()
ds_train = common.label_encode(ds_train)
ds_test = common.label_encode(ds_test)

ds_train = ds_train.fillna(-1)
ds_test = ds_test.fillna(-1)


df_X = ds_train.loc[:, ds_train.columns != 'target']
df_y = ds_train['target']

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33, random_state=config.rand_state)

ss = StandardScaler()
X_train_scalar = pd.DataFrame(ss.fit_transform(X_train), columns = X_train.columns)
X_test_scalar = pd.DataFrame(ss.fit_transform(X_test), columns = X_test.columns)



NameError: name 'StandardScaler' is not defined

(scheduler +4m32s) Resized to 21 CPUs.


In [15]:
X_train_id = ray.put(X_train_scalar)
y_train_id = ray.put(y_train)

X_test_id = ray.put(X_test_scalar)
y_test_id = ray.put(y_test)


NameError: name 'X_train' is not defined

In [ ]:

@ray.remote
def worker(base_model, X_train_id, X_test_id, y_train_id, y_test_id):     
    base_model.fetch_model(X_train_id, X_test_id, y_train_id, y_test_id)
    return base_model


In [ ]:
brisk_xgb1 = BriskXGBoost('brisk_xgb1')
brisk_xgb1.boosted_round = 100
brisk_xgb1.n_trials = 500

brisk_xgb2 = BriskXGBoost('brisk_xgb2')
brisk_xgb2.boosted_round = 10
brisk_xgb2.n_trials = 10

base_models = [brisk_xgb1, brisk_xgb2]

brisk_bagging_1 = BriskBagging('brisk_bagging_1')
brisk_bagging_1.boosted_round = 100
brisk_bagging_1.n_trials = 50

brisk_bagging_2 = BriskBagging('brisk_bagging_2')
brisk_bagging_2.boosted_round = 100
brisk_bagging_2.n_trials = 50

base_models_bagging = [brisk_bagging_1, brisk_bagging_2]

slug_xgb1 = SlugXGBoost('slug_xgb1')
slug_xgb1.boosted_round = 100
slug_xgb1.n_trials = 500

slug_xgb2 = SlugXGBoost('slug_xgb2')
slug_xgb2.boosted_round = 100
slug_xgb2.n_trials = 500

base_models_slug = [slug_xgb1, slug_xgb2]

#slug_ann_1 = SlugANN('slug_ann_1', X_train, X_test, y_train, y_test)
#slug_ann_1.epochs = 50
#slug_ann_1.n_trials = 50

#slug_ann_2 = SlugANN('slug_ann_2', X_train, X_test, y_train, y_test)
#slug_ann_2.epochs = 50
#slug_ann_2.n_trials = 50

#base_models_ann = [slug_ann_1, slug_ann_2]


slug_rf_1 = SlugRF('slug_rf_1')
slug_rf_1.max_n_estimators = 1000
slug_rf_1.n_trials = 50

slug_rf_2 = SlugRF('slug_rf_2')
slug_rf_2.max_n_estimators = 1000
slug_rf_2.n_trials = 50

base_models_rf = [slug_rf_1, slug_rf_2]


slug_knn_1 = SlugKNN('slug_knn_1')
slug_knn_1.n_neighbors = 100
slug_knn_1.n_trials = 500

slug_knn_2 = SlugKNN('slug_knn_2')
slug_knn_2.n_neighbors = 100
slug_knn_1.n_trials = 500

base_models_knn = [slug_knn_1, slug_knn_2]


In [ ]:
model_results = ray.get([worker.remote(base_model, X_train_id, X_test_id, y_train_id, y_test_id) for base_model in base_models])

In [ ]:
model_results[0].score

(scheduler +3m45s) Resized to 27 CPUs.
(scheduler +3m45s) Removing 2 nodes of type small-group (idle).
(scheduler +3m53s) Resized to 23 CPUs.
(scheduler +3m53s) Removing 2 nodes of type small-group (idle).
(scheduler +4m0s) Resized to 19 CPUs.
(scheduler +4m8s) Resized to 15 CPUs.
(scheduler +4m8s) Removing 1 nodes of type small-group (idle).
(scheduler +4m16s) Resized to 17 CPUs.
(scheduler +4m24s) Removing 1 nodes of type small-group (idle).


In [12]:
model_results[1].score

[0.9643010946844719, 0.8669560346764794]

In [13]:
model_results[1].predict(X_test)

array([220875.        , 150083.33333333, 103940.25      , 228575.        ,
        86816.66666667, 110866.66666667, 272666.66666667, 124927.83333333,
       580430.16666667, 159475.16666667, 193922.91666667, 149821.5       ,
       233342.5       , 123408.33333333, 132937.5       , 149257.25      ,
       226194.        , 120783.33333333, 149816.66666667, 246570.83333333,
       129862.5       , 142083.33333333, 101387.41666667, 162816.66666667,
       191054.16666667, 217902.91666667, 174527.91666667,  85533.33333333,
       306740.33333333, 114223.        , 134266.66666667, 184091.66666667,
       141720.83333333, 312320.83333333, 333725.75      , 179391.66666667,
       278575.        , 121750.        , 236150.83333333, 296191.66666667,
       189833.33333333, 117450.        , 180413.25      , 304943.83333333,
       324655.08333333, 148150.        , 126025.        , 118530.5       ,
       172438.66666667,  98395.83333333, 363839.5       , 143241.66666667,
       166112.5       ,  